### 1. 데이터 변환
(1) 표준화 (이상치처리)  
(2) 정규화 (0~1)  
(3) 이산화(Binarizer): 연속형 -> 이항변수, (Binarizer, OneHot Encoding)  
(4) 변수 개수 축소(feature selection)  
(5) 차원축소 (PCA)    
(6) 시그널 변환(퓨리에변환, 웨이블릿 변환)   

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={"axes.facecolor":"white", "axes.edgecolor":"black"})
import matplotlib as mpl
from sklearn.decomposition import PCA

### 데이터 정규화 전/후 결과 비교

In [59]:
# 맨 마지막 열이 label
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

In [60]:
# scaling 식 직접 쓰기
# 타겟은 scaling 할 필요 없음
def MyScaler(data): #MinMaxScaling
    # print(np.mIN(data)) # 전체데이터에서 최소값
    de=np.max(data, axis=0)-np.min(data,axis=0) # 열 단위 최소값 (각 feature를 정규화해야하기 때문에 열 단위로)
    num = data-np.min(data,axis=0) # 각 데이터에서 해당 열의 최소값을 뺸 값 출력
    return num/de
    # print(np.min(data, axis=1)) # 행 단위 최소값

In [62]:
# 정규화
xy=MyScaler(xy)
# xy.shape #(8, 5)
print(xy)

[[1.         1.         0.         1.         1.        ]
 [0.70548491 0.70439552 1.         0.71881783 0.83755792]
 [0.54412549 0.50274824 0.57608696 0.60646801 0.6606331 ]
 [0.33890353 0.31368023 0.10869565 0.45989134 0.43800918]
 [0.51436    0.4258239  0.30434783 0.58504805 0.42624401]
 [0.49556179 0.4258239  0.31521739 0.48131134 0.49276137]
 [0.11436064 0.         0.20652174 0.22007776 0.18597238]
 [0.         0.07747099 0.5326087  0.         0.        ]]


In [50]:
xdata=xy[:,0:-1] # 맨 마지막 열을 제외한 열들 추출
xdata.shape

(8, 4)

In [51]:
ydata=xy[:,[-1]] # 맨 마지막 열 추출
ydata.shape

(8, 1)

In [52]:
x=tf.placeholder(tf.float32, shape=[None,4])
y=tf.placeholder(tf.float32, shape=[None,1])
w=tf.Variable(tf.random_normal([4,1]), name="weight")
b=tf.Variable(tf.random_normal([1]), name="bias")

In [53]:
hf=tf.matmul(x,w)+b
cost=tf.reduce_mean(tf.square(hf-y))

In [56]:
train=tf.train.GradientDescentOptimizer(1e-5).minimize(cost)

In [57]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(101):
    cv,hv,_=sess.run([cost,hf,train], feed_dict={x:xdata, y:ydata})
    print(step,"cost:",cv, "hf:",hv)

0 cost: 1.5128553 hf: [[-1.1081699 ]
 [-1.0915207 ]
 [-0.6612911 ]
 [-0.1562188 ]
 [-0.49465302]
 [-0.49340346]
 [ 0.1430142 ]
 [ 0.11878243]]
1 cost: 1.5127568 hf: [[-1.1081103 ]
 [-1.0914645 ]
 [-0.66124475]
 [-0.15618348]
 [-0.49461055]
 [-0.4933626 ]
 [ 0.14303996]
 [ 0.11880726]]
2 cost: 1.5126586 hf: [[-1.1080507 ]
 [-1.0914084 ]
 [-0.6611985 ]
 [-0.15614817]
 [-0.49456808]
 [-0.49332175]
 [ 0.14306575]
 [ 0.1188321 ]]
3 cost: 1.51256 hf: [[-1.1079912 ]
 [-1.0913522 ]
 [-0.6611522 ]
 [-0.15611285]
 [-0.4945256 ]
 [-0.49328095]
 [ 0.1430915 ]
 [ 0.11885694]]
4 cost: 1.5124615 hf: [[-1.1079315 ]
 [-1.091296  ]
 [-0.66110575]
 [-0.15607753]
 [-0.49448314]
 [-0.4932401 ]
 [ 0.14311728]
 [ 0.11888176]]
5 cost: 1.5123632 hf: [[-1.107872  ]
 [-1.0912399 ]
 [-0.6610594 ]
 [-0.15604222]
 [-0.49444067]
 [-0.49319923]
 [ 0.14314304]
 [ 0.1189066 ]]
6 cost: 1.5122647 hf: [[-1.1078123 ]
 [-1.0911838 ]
 [-0.661013  ]
 [-0.1560069 ]
 [-0.4943982 ]
 [-0.49315837]
 [ 0.14316879]
 [ 0.11893143]]
7

### Softmax 함수

In [70]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
#y값은 one hot 인코딩 방식으로 초기화 함...(a,b,c)중 하나를 hot하게 함...
y_data = [[0, 0, 1],#2
          [0, 0, 1],#2
          [0, 0, 1],#2
          [0, 1, 0],#1
          [0, 1, 0],#1
          [0, 1, 0],#1
          [1, 0, 0],#0
          [1, 0, 0]]#0

In [75]:
num_classes=3
x=tf.placeholder("float",[None,4])
y=tf.placeholder("float",[None,num_classes])
w=tf.Variable(tf.random_normal([4,num_classes]))
b=tf.Variable(tf.random_normal([num_classes]))
#hf=x*w+b
#[None,4],[4,3]+[3]=

In [79]:
hf=tf.nn.softmax(tf.matmul(x,w)+b) # 합이 1인 확률로 출력하기위해 softmax 함수 적용
cost= tf.reduce_mean(-tf.reduce_sum(y*tf.log(hf), axis=1)) 
train=tf.train.GradientDescentOptimizer(0.1).minimize(cost)

In [103]:
# with 구문을 벗어나면 session 객체가 해제됨
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(2001):
        sess.run(train, feed_dict={x:x_data, y:y_data})
        if step %200 == 0:
            print(step, sess.run(cost,feed_dict={x:x_data, y:y_data}))
    print("="*50)
# xdata= 1,11,7,9 => y?
    res=sess.run(hf, feed_dict={x:[[1,11,7,9]]})
    # 본래의 x데이터가 2차원이므로 xdata를 2차원으로 써줘야함
    print(sess.run(tf.argmax(res,axis=1)))
    print("="*50)
    res=sess.run(hf, feed_dict={x:[[1,11,7,9],
                                   [1,3,4,3],
                                   [1,1,0,1]]})
    print(sess.run(tf.argmax(res,axis=1))) # 함수를 최대값으로 만들기 위한 인덱스 출력

0 1.6480659
200 0.506062
400 0.4128444
600 0.33731052
800 0.2632463
1000 0.22669907
1200 0.2056907
1400 0.1881704
1600 0.17332926
1800 0.16059835
2000 0.1495614
[1]
[1 0 2]


In [118]:
xy=np.loadtxt("data/zoo.csv", delimiter=',', dtype=np.float32)
xy.shape #(101, 17)
xdata=xy[:,0:-1]
ydata=xy[:,[-1]]
print(xdata.shape)
print(ydata.shape)

(101, 16)
(101, 1)


In [131]:
x=tf.placeholder(tf.float32, [None,16])
y=tf.placeholder(tf.int32, [None,1]) 
y_one_hot=tf.one_hot(y,7) # (원핫인코딩할 변수, 분류되어야할 종류), ex) 동물이 3종류 : 3(y) -> 0001000(y_one_hot)
print("one_hot", y_one_hot)
y_one_hot=tf.reshape(y_one_hot,[-1,7])
print("one_hot after reshape", y_one_hot)

one_hot Tensor("one_hot_5:0", shape=(?, 1, 7), dtype=float32)
one_hot after reshape Tensor("Reshape_2:0", shape=(?, 7), dtype=float32)


one hot 함수는 한 차원 높게 변환됨  
y : [[0],[3],....,[5]] => [None,1] 의 shape  
one_hot => [[[1000000]],[[0001000]],....,[[00000100]]] => [None,1,7]  
우리가 바라는 출력 결과의 모습은 [None,7] 임  
이를 위해서, reshape 을 하면 됨 tf.shape(y_one_hot,[-1,7]) 코딩하면 [None,7]로 reshape 됨  
[[[1000000]],[[0001000]]]...] => [[1000000],[0001000]....]  

y : 0~6 (7가지 종류의 동물) -> 분류기 7개  
w1x1+w2x2+w3x3...w7x7+b => softmax -> 0번 종류의 동물에 대한 확률  
w1x1+w2x2+w3x3...w7x7+b => softmax -> 1번 종류의 동물에 대한 확률  
....  
w1x1+w2x2+w3x3...w7x7+b => softmax -> 6번 종류의 동물에 대한 확률  
편하게 쓰기 위해 매트릭스 활용  
y데이터가 원핫인코딩되어 있는 경우, 1000000, 0100000... => [None,7]   

In [138]:
w=tf.Variable(tf.random_normal([16,7]))
b=tf.Variable(tf.random_normal([7]))
# x:[None,16], w:[16,7]
logit=tf.matmul(x,w)+b
hf=tf.nn.softmax(logit) 
cost=tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=y_one_hot)
cost2=tf.reduce_mean(cost)
train=tf.train.GradientDescentOptimizer(0.1).minimize(cost2)
prediction=tf.argmax(hf,1) # 최대값에 해당하는 인덱스가 출력
# 예측이 실제값과 잘 맞는지 노드 정의
correct_prediction=tf.equal(prediction,tf.argmax(y_one_hot,1))
# 정확도를 수치로 표현
accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [146]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train,feed_dict={x:xdata,y:ydata})
        if step % 100 ==0:
            cv,av=sess.run([cost2, accuracy], feed_dict={x:xdata,y:ydata})
            print("step:",step,"cost:",cv,"acc:",av)

# 전체 데이터로 트레이닝 수행하여 모델 생성
# 모델에 전체 데이터를 집어 넣어서 정확도 출력
# 데이터를 분할하지 않았음
    pred=sess.run(prediction, feed_dict={x:xdata})
    for p,y in zip(pred, ydata.flatten()):
          print("[{}] 예측: {} 실제 y: {}".format(p==int(y),p,int(y)))
    
    # print(sess.run(prediction, feed_dict={x:[[...]]}))
    # [[0],[3],....] => [0,3,...]
    # print(pred) # 우리 모델에서 예측한 클래스 
    
    # flatten : 차원을 한차원 내려주는 역할
    # [[1],[0]] -> [1.0]

step: 0 cost: 5.9643197 acc: 0.0990099
step: 100 cost: 0.6303851 acc: 0.8217822
step: 200 cost: 0.41774717 acc: 0.8811881
step: 300 cost: 0.3232745 acc: 0.9108911
step: 400 cost: 0.26422137 acc: 0.9207921
step: 500 cost: 0.22188602 acc: 0.9405941
step: 600 cost: 0.18959822 acc: 0.9405941
step: 700 cost: 0.16425446 acc: 0.95049506
step: 800 cost: 0.14406475 acc: 0.980198
step: 900 cost: 0.12780881 acc: 0.980198
step: 1000 cost: 0.11458009 acc: 0.980198
step: 1100 cost: 0.10369035 acc: 0.990099
step: 1200 cost: 0.09461895 acc: 0.990099
step: 1300 cost: 0.086974174 acc: 0.990099
step: 1400 cost: 0.080460854 acc: 1.0
step: 1500 cost: 0.07485528 acc: 1.0
step: 1600 cost: 0.069986284 acc: 1.0
step: 1700 cost: 0.06572138 acc: 1.0
step: 1800 cost: 0.061956957 acc: 1.0
step: 1900 cost: 0.05861109 acc: 1.0
step: 2000 cost: 0.05561837 acc: 1.0
step: 2100 cost: 0.05292597 acc: 1.0
step: 2200 cost: 0.050490987 acc: 1.0
step: 2300 cost: 0.048278067 acc: 1.0
step: 2400 cost: 0.04625815 acc: 1.0
step:

#### 연습문제
1. mushroom 데이터 분류 모델
2. animal 데이터를 7:3로 분할 -> 모델, 테스트
3. breask cancer(유방암) 분류기
4. wine quality 분류기 (10단계) 
데이터:https://archive.ics.uci.edu/ml/datasets.php?format=&task=&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table